<a href="https://colab.research.google.com/github/itsannapanna/Donor-predictor-BloodLink/blob/main/machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Step 1: Load and Preprocess the Data
train_data = pd.read_csv('/content/blood-train (2).csv')
test_data = pd.read_csv('/content/test.csv')

# Assuming '0' means ineligible and '1' means eligible
train_data['Eligibility'] = (train_data['Months since Last Donation'] > 6).astype(int)
test_data['Eligibility'] = (test_data['Months since Last Donation'] > 6).astype(int)

# Encoding categorical variable (Blood Group)
train_data = pd.get_dummies(train_data, columns=['Blood Group'])
test_data = pd.get_dummies(test_data, columns=['Blood Group'])
train_data.drop(columns=['Blood Group_A','Made Donation in March 2007'], inplace=True)


# Step 2: Train the Model


In [67]:
RFC = RandomForestClassifier()

X_train = train_data.drop(['Name of the donor', 'Eligibility'], axis=1)
y_train = train_data['Eligibility']

In [68]:
RFC.fit(X_train, y_train)

RandomForestClassifier()

In [69]:
X_train

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Blood Group_A+,Blood Group_A-,Blood Group_AB+,Blood Group_AB-,Blood Group_B+,Blood Group_B-,Blood Group_O+,Blood Group_O-
0,619,2,50,12500,98,0,0,0,0,0,1,0,0
1,664,0,13,3250,28,0,0,0,0,0,0,0,1
2,441,1,16,4000,35,0,0,0,1,0,0,0,0
3,160,2,20,5000,45,0,0,0,0,1,0,0,0
4,358,1,24,6000,77,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,698,23,1,250,23,1,0,0,0,0,0,0,0
572,433,16,3,750,86,0,0,0,0,0,0,1,0
573,360,21,2,500,52,0,0,0,0,0,1,0,0
574,541,39,1,250,39,0,1,0,0,0,0,0,0


In [70]:
# Step 3: Preprocess and Predict
test_data_processed = test_data.drop(columns=['Name of the donor'])

feature_names_train = X_train.columns.tolist()
feature_names_test = test_data_processed.columns.tolist()

# Identify missing features in the test data
missing_features = list(set(feature_names_train) - set(feature_names_test))

# Add missing features to the test data
for feature in missing_features:
    test_data_processed[feature] = 0

# Reorder columns to match training data
test_data_processed = test_data_processed[feature_names_train]


# Assuming you have already instantiated a StandardScaler 'scale'
scale = StandardScaler()
scale.fit(X_train)
# Fit and transform the provided data using the scaler
test_data_scaled = scale.transform(test_data_processed)


In [71]:
missing_features

[]

In [72]:
# Make predictions
predicted_probabilities = RFC.predict_proba(test_data_scaled)[:, 1]

# Step 4: Define Threshold and Classify
threshold = 0.5  # Adjust as needed
predicted_labels = (predicted_probabilities > threshold).astype(int)

# Step 5: Filter and Retrieve Corresponding Donor Names
eligible_donor_names = test_data.loc[predicted_labels == 1, 'Name of the donor']

# Step 6: Print the Resulting Donor Names
print("\nEligible Donor Names:")
print(eligible_donor_names)


Eligible Donor Names:
Series([], Name: Name of the donor, dtype: object)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [74]:
predicted_probabilities = RFC.predict_proba(test_data_scaled)[:, 1]

# Step 4: Define Threshold and Classify
threshold = 0.5
predicted_labels = (predicted_probabilities > threshold).astype(int)

blood_group_input = 'B+'  # Change this to the desired blood group
eligible_indices = (test_data['Blood Group_' + blood_group_input] == 1)
eligible_donor_names = test_data.loc[eligible_indices, 'Name of the donor']

# Print the Resulting Donor Names
print("\nEligible Donor Names for Blood Group", blood_group_input + ":")
print(eligible_donor_names)


Eligible Donor Names for Blood Group A+:
4        Meera
9        Kabir
28        Avni
33       Shaan
36     Akshara
40      Suhana
45       Arnav
53       Kabir
57      Vihaan
68       Navya
72        Zoya
85       Arnav
89     Atharva
97       Kabir
105     Vihaan
113      Aryan
125     Pranav
137    Reyansh
141     Advait
149      Krish
157      Arjun
169      Ayaan
177      Kabir
181      Advik
189       Aadi
Name: Name of the donor, dtype: object


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [77]:
import pickle
from sklearn.ensemble import RandomForestClassifier

# Train your RandomForestClassifier model
RFC = RandomForestClassifier()
RFC.fit(X_train, y_train)

# Serialize the model using pickle and save it to a file
with open('model.pkl', 'wb') as f:
    pickle.dump(RFC, f)

In [83]:
from flask import Flask, request, jsonify
import pickle

app = Flask(__name__)

# Load the serialized model
with open('model.pkl', 'rb') as f:
    RFC = pickle.load(f)

# Define the prediction endpoint



 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
predictions = model.predict(new_data)
